In [2]:
import pandas as pd
import numpy as np
import src.Cleaning_functions as fn

In [3]:
import h2o
from h2o.automl import H2OAutoML

Initiate the h2o:

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_252"; OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09); OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
  Starting server from /home/david/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppxo78w1c
  JVM stdout: /tmp/tmppxo78w1c/h2o_david_started_from_python.out
  JVM stderr: /tmp/tmppxo78w1c/h2o_david_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_david_ydbcp9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.710 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


Import and clean the data:

In [48]:
train = pd.read_csv('./INPUT/diamonds_train.csv') 


In [80]:
features = ['carat','table','depth', 'x', 'y', 'z']


new_df = pd.DataFrame()
for e in features:
    name = e + '_log'
    new_df[name] = train[e].apply(lambda x: np.log1p(x))
new_train = train.drop(features, axis=1).reset_index(drop=True)
train_clean = pd.concat([new_train, new_df], axis=1, sort=False)

#Categories to numericals:
train_clean.replace({'Premium': 1, 'Ideal': 2, 'Very Good':3, 'Good':4, 'Fair':5}, inplace=True)
train_clean.replace({'D':1, 'E':2, 'F':3, 'G':4, 'H':5, 'I':6, 'J':7}, inplace=True)
train_clean.replace({'IF':1, 'VVS1':2, 'VVS2':3, 'VS1':4, 'VS2':5, 'SI1':6, 'SI2':7, 'I1':8}, inplace=True)

#Standardize columns:
train_clean = fn.standardize(train_clean,['carat_log','table_log','x_log','y_log'] )

#Unuseful columns
train_clean.drop(['Unnamed: 0','depth_log','z_log'], axis= 1, inplace=True)

Parse the csv to a h2o object:

In [82]:
trainh2o = h2o.H2OFrame(train_clean)
#train, valid = trainh2o.split_frame(ratios = [.8], seed = 1234)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [83]:
x = trainh2o.columns
y = "price"
x.remove(y)

Train the auto ML model for one hour:

In [84]:
aml = H2OAutoML(seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


Check the leaderboard:

In [85]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200511_190504,284059,532.972,284059,272.844,0.0982167
StackedEnsemble_BestOfFamily_AutoML_20200511_190504,285240,534.079,285240,275.518,0.0997747
GBM_2_AutoML_20200511_190504,290943,539.391,290943,279.494,nan
GBM_3_AutoML_20200511_190504,291772,540.16,291772,277.776,0.0988642
GBM_4_AutoML_20200511_190504,293897,542.123,293897,273.687,nan
GBM_1_AutoML_20200511_190504,294825,542.978,294825,284.007,0.103185
GBM_grid__1_AutoML_20200511_190504_model_4,296907,544.891,296907,273.279,0.092564
GBM_grid__1_AutoML_20200511_190504_model_5,298056,545.945,298056,274.818,0.0959358
GBM_grid__1_AutoML_20200511_190504_model_28,299711,547.459,299711,278.64,0.0949812
GBM_grid__1_AutoML_20200511_190504_model_2,300107,547.821,300107,282.425,0.100787


In [86]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200511_190504

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 193669.96943699953
RMSE: 440.07950354112097
MAE: 237.29442693662838
RMSLE: 0.09200751941573657
R^2: 0.9877616234038199
Mean Residual Deviance: 193669.96943699953
Null degrees of freedom: 32389
Residual degrees of freedom: 32377
Null deviance: 512565556449.8945
Residual deviance: 6272970310.064415
AIC: 486259.8090225412

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 284058.8724876938
RMSE: 532.9717370439954
MAE: 272.8441730954824
RMSLE: 0.09821666547882263
R^2: 0.9820497753621964
Mean Residual Deviance: 284058.8724876938
Null degrees of freedom: 32389
Residual degrees of freedom: 32376
Null deviance: 512617094235.09143
Residual deviance: 9200666879.876402
AIC: 498668.0209111941


Save the model:

In [119]:
model_saved = h2o.save_model(model=aml.leader, path="../OUTPUT/mymodelh2o", force=True)

Prepare test data:

In [94]:
test = pd.read_csv('./INPUT/diamonds_test.csv') 

features = ['carat','table','depth', 'x', 'y', 'z']


new_df = pd.DataFrame()
for e in features:
    name = e + '_log'
    new_df[name] = test[e].apply(lambda x: np.log1p(x))
new_test = test.drop(features, axis=1).reset_index(drop=True)
test_clean = pd.concat([new_test, new_df], axis=1, sort=False)

#Categories to numericals:
test_clean.replace({'Premium': 1, 'Ideal': 2, 'Very Good':3, 'Good':4, 'Fair':5}, inplace=True)
test_clean.replace({'D':1, 'E':2, 'F':3, 'G':4, 'H':5, 'I':6, 'J':7}, inplace=True)
test_clean.replace({'IF':1, 'VVS1':2, 'VVS2':3, 'VS1':4, 'VS2':5, 'SI1':6, 'SI2':7, 'I1':8}, inplace=True)

#Standardize columns:
test_clean = fn.standardize(test_clean,['carat_log','table_log','x_log','y_log'] )

#Unuseful columns
test_clean.drop(['Unnamed: 0','depth_log','z_log'], axis= 1, inplace=True)

Parse to h2o object

In [95]:
testh2o = h2o.H2OFrame(test_test)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [96]:
preds = aml.leader.predict(testh2o)


stackedensemble prediction progress: |████████████████████████████████████| 100%


Prepare the predictions to a csv:

In [112]:
preds_df = h2o.as_list(preds)

In [113]:
predict = pd.DataFrame({'id':list(range(len(test))), 'price':preds_df['predict']})

In [118]:
predict.to_csv('./OUTPUT/submission.csv', index=False)